In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
# output
out_f <- '@@@@@/sex-div-analysis/phe_data/v2.1-no-covars/Testosterone.phe'

# phenotype file
phe_dir <- '@@@@@/biomarkers/covariate_corrected'
phe_f   <- file.path(phe_dir, 'phenotypes/biomarkers_with_egfr_fastingglucose_nonalbumin.phe')
covar_f <- file.path(phe_dir, 'outputExtendedNoTDIreduced/phenotypes/full.table.combined.phe')
gwas_covar_f <- '@@@@@/ukbb24983/sqc/population_stratification_w24983_20200313/ukb24983_GWAS_covar.20200313.phe'

# covar list
covars_lst <- '@@@@@/sex-div-analysis/snpnet/phe_data/v2/covars.sex-indep.txt'

# population definition
pop_def_d <- dirname(covars_lst)
pop_defs <- c('combined', 'onesex', 'zerosex')


In [3]:
gwas_covar_df <- fread(
    gwas_covar_f,
    select=c('FID', 'IID', 'split', 'age', 'sex'),
    colClasses=c('FID'='character', 'IID'='character')
)


In [4]:
pop_def_dfs <- list()
for(pop in pop_defs){
    pop_def_dfs[[pop]] <- file.path(pop_def_d, paste0(pop, '.phe')) %>%
    fread(select=c('FID','IID'),colClasses=c('FID'='character', 'IID'='character'))
}


In [5]:
covars <- fread(covars_lst, head=F) %>% pull()

In [6]:
covar_df <- fread(covar_f, colClasses=c('FID'='character', 'IID'='character')) %>% 
mutate(ageBin_FastingTime = ageBin * FastingTime) %>%
select(c('FID', 'IID', all_of(covars)))


In [7]:
df <- fread(phe_f, select=c('IID', 'f.30850.0.0'), colClasses=c('IID'='character')) %>%
rename('Testosterone'='f.30850.0.0')%>%
mutate(FID = IID) %>%
select(FID, IID, Testosterone) %>%
mutate(Testosterone = log10(Testosterone)) %>%
left_join(covar_df, by=c('FID', 'IID')) %>%
left_join(gwas_covar_df, by=c('FID', 'IID')) %>%
mutate(
    split2 = if_else(is.na(Testosterone), '', split),
    split_combined = if_else(IID %in% (pop_def_dfs[['combined']] %>% pull(FID)), split2, '-'),
    split_onesex   = if_else(IID %in% (pop_def_dfs[['onesex']] %>% pull(FID)), split2, '-'),    
    split_zerosex  = if_else(IID %in% (pop_def_dfs[['zerosex']] %>% pull(FID)), split2, '-')
)


In [8]:
df %>% 
colnames()

[1] "FID"                    "IID"                    "Testosterone"          
  [4] "PC1"                    "PC2"                    "PC3"                   
  [7] "PC4"                    "PC5"                    "PC6"                   
 [10] "PC7"                    "PC8"                    "PC9"                   
 [13] "PC10"                   "PC11"                   "PC12"                  
 [16] "PC13"                   "PC14"                   "PC15"                  
 [19] "PC16"                   "PC17"                   "PC18"                  
 [22] "PC19"                   "PC20"                   "PC21"                  
 [25] "PC22"                   "PC23"                   "PC24"                  
 [28] "PC25"                   "PC26"                   "PC27"                  
 [31] "PC28"                   "PC29"                   "PC30"                  
 [34] "PC31"                   "PC32"                   "PC33"                  
 [37] "PC34"                   "PC35"                   "PC36"                  
 [40] "PC37"                   "PC38"                   "PC39"                  
 [43] "PC40"                   "UrineSampleMinutes"     "DrawTime"              
 [46] "DilutionFactorTimeZero" "FastingTime"            "f.53.0.0_2006"         
 [49] "f.53.0.0_2007.04"       "f.53.0.0_2007.05"       "f.53.0.0_2007.06"      
 [52] "f.53.0.0_2007.07"       "f.53.0.0_2007.08"       "f.53.0.0_2007.09"      
 [55] "f.53.0.0_2007.10"       "f.53.0.0_2007.11"       "f.53.0.0_2007.12"      
 [58] "f.53.0.0_2008.01"       "f.53.0.0_2008.02"       "f.53.0.0_2008.03"      
 [61] "f.53.0.0_2008.04"       "f.53.0.0_2008.05"       "f.53.0.0_2008.06"      
 [64] "f.53.0.0_2008.07"       "f.53.0.0_2008.08"       "f.53.0.0_2008.09"      
 [67] "f.53.0.0_2008.10"       "f.53.0.0_2008.11"       "f.53.0.0_2008.12"      
 [70] "f.53.0.0_2009.01"       "f.53.0.0_2009.02"       "f.53.0.0_2009.03"      
 [73] "f.53.0.0_2009.04"       "f.53.0.0_2009.05"       "f.53.0.0_2009.06"      
 [76] "f.53.0.0_2009.07"       "f.53.0.0_2009.08"       "f.53.0.0_2009.09"      
 [79] "f.53.0.0_2009.10"       "f.53.0.0_2009.11"       "f.53.0.0_2009.12"      
 [82] "f.53.0.0_2010.01"       "f.53.0.0_2010.02"       "f.53.0.0_2010.03"      
 [85] "f.53.0.0_2010.04"       "f.53.0.0_2010.05"       "f.53.0.0_2010.06"      
 [88] "f.53.0.0_2010.07"       "f.53.0.0_2010.0810"     "f.54.0.0_10003"        
 [91] "f.54.0.0_11001"         "f.54.0.0_11002"         "f.54.0.0_11003"        
 [94] "f.54.0.0_11004"         "f.54.0.0_11005"         "f.54.0.0_11006"        
 [97] "f.54.0.0_11007"         "f.54.0.0_11008"         "f.54.0.0_11009"        
[100] "f.54.0.0_11010"         "f.54.0.0_11011"         "f.54.0.0_11012"        
[103] "f.54.0.0_11013"         "f.54.0.0_11014"         "f.54.0.0_11016"        
[106] "f.54.0.0_11017"         "f.54.0.0_11018"         "f.54.0.0_11020"        
[109] "f.54.0.0_11021"         "f.54.0.0_11022"         "f.54.0.0_11023"        
[112] "ageIndicator"           "ageBin"                 "Batch"                 
[115] "Ethnicity"              "ageBin_FastingTime"     "split"                 
[118] "age"                    "sex"                    "split2"                
[121] "split_combined"         "split_onesex"           "split_zerosex"

In [9]:
df %>%
count(split, split_combined, split_onesex, split_zerosex) %>%
drop_na(split_combined) %>%
filter(split_combined != '')

split,split_combined,split_onesex,split_zerosex,n
<chr>,<chr>,<chr>,<chr>,<int>
test,-,-,-,10189
test,test,-,test,28640
test,test,test,-,28601
train,-,-,-,35527
train,train,-,train,99563
train,train,train,-,100913
val,-,-,-,5071
val,val,-,val,14049
val,val,val,-,14594


In [11]:
df %>%
count(split, split_combined, split_onesex, split_zerosex) %>%
drop_na(split_combined) %>%
filter(split_combined != '-')

split,split_combined,split_onesex,split_zerosex,n
<chr>,<chr>,<chr>,<chr>,<int>
test,test,-,test,28640
test,test,test,-,28601
train,train,-,train,99563
train,train,train,-,100913
val,val,-,val,14049
val,val,val,-,14594


In [13]:
df %>%
drop_na(split_combined) %>%
filter(split_combined != '-') %>%
fwrite(out_f, sep='\t', na = "NA", quote=F)
